In [1]:
from TwitterAPI import TwitterAPI
import requests
from bs4 import BeautifulSoup
import re
import sys
import time
import os
import numpy as np
try:
    import configparser
except:
    from six.moves import configparser

In [2]:
def getCandidates(path):
    r = requests.get("http://twittercounter.com/pages/100?vt=1&utm_expid=102679131-111.l9w6V73qSUykZciySuTZuA.1&utm_referrer=https%3A%2F%2Fwww.google.com%2F")
    soup = BeautifulSoup(r.text, "lxml")
    lst=[]
    f=open(path+"/candidates.txt","w")
    for i in soup.find_all("div", attrs={"class": "name-bio"}):
        tagSoup=BeautifulSoup(str(i), "lxml")
        for j in tagSoup.find_all("span", attrs={"itemprop": "alternateName"}):
            f.write(re.sub(r"@","",j.contents[0])+"\n")
    f.close()

In [3]:
# This method is done for you. Make sure to put your credentials in the file twitter.cfg.
def get_twitter(config_file):
    """ Construct an instance of TwitterAPI using the tokens you entered above.
    Returns:
      An instance of TwitterAPI.
    """     
    config = configparser.ConfigParser()
    config.read(config_file)
    twitter = TwitterAPI(
                   config.get('twitter', 'consumer_key'),
                   config.get('twitter', 'consumer_secret'),
                   config.get('twitter', 'access_token'),
                   config.get('twitter', 'access_token_secret'))
    return twitter

In [4]:
# I've provided the method below to handle Twitter's rate limiting.
# You should call this method whenever you need to access the Twitter API.
def robust_request(twitter, resource, params, max_tries=5):
    """ If a Twitter request fails, sleep for 15 minutes.
    Do this at most max_tries times before quitting.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request; e.g., "friends/ids"
      params ..... A parameter dict for the request, e.g., to specify
                   parameters like screen_name or count.
      max_tries .. The maximum number of tries to attempt.
    Returns:
      A TwitterResponse object, or None if failed.
    """
    for i in range(max_tries):
        request = twitter.request(resource, params)
        if request.status_code == 200:
            return request
        else:
            print('Got error %s \nsleeping for 15 minutes.' % request.text)
            sys.stderr.flush()
            time.sleep(61 * 15)

In [5]:
def get_friends(twitter, screen_name):
    """ Return a list of Twitter IDs for users that this person follows, up to 5000.
    See https://dev.twitter.com/rest/reference/get/friends/ids

    Note, because of rate limits, it's best to test this method for one candidate before trying
    on all candidates.

    Args:
        twitter.......The TwitterAPI object
        screen_name... a string of a Twitter screen name
    Returns:
        A list of ints, one per friend ID, sorted in ascending order.

    """
    ###TODO

    resource='friends/list'
    params = {'screen_name': screen_name, 'count': 5000}
    req=robust_request(twitter, resource, params, max_tries=5)
    
    f=open("data/user_friends.txt","a")
    for r in req:
        f.write("\n"+screen_name+" "+r['screen_name'])
    f.close()

In [ ]:
def getMovieReviews(url):
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    cnt=0
    lst=[]
    flag=True
    f=open("data/movie_reviews.txt","w")
    
    for i in soup.find_all("tr", attrs={"itemprop": "itemListElement"}):
        
        j=i.find_all("a", attrs={"class": "unstyled articleLink"})

        r1 = requests.get("https://www.rottentomatoes.com"+j[0]['href'])
        soup1 = BeautifulSoup(r1.text, "lxml")
            
        for k in soup1.find_all("div", attrs={"id":"reviews"}):
            for l in k.find_all("p"):
                if flag:
                    f.write(l.text.lstrip())
                    flag=False
                else:
                    f.write("\n"+l.text.lstrip())
    f.close()

In [ ]:
def main():
    """
    FYI: This takes ~50-60 mins to run on my laptop.
    """
    startTime = time.clock()
    if not os.path.exists('data'):
        os.makedirs('data')
        
##########-----------------Data for classifier-------------############      
    # Get Movoie reviews from the https://www.rottentomatoes.com/

    getMovieReviews("https://www.rottentomatoes.com/top/bestofrt/")
    
    print("Movie Reviews Collected")
##########-----------------Data for cluster----------------############
    # download http://twittercounter.com/
    # Get top 100 twitter candidates
    getCandidates("data")
    
    print("Candidates Collected")
    #get twitter object
    twitter=get_twitter('twitter.cfg')
  
    #get candidate names for getting the candidate's friends
    screen_names=[]
    for f in open("data/candidates.txt","r"):
        screen_names.append(f.strip('\n'))
    
    for i,item in enumerate(screen_names):
        get_friends(twitter,item)
    
    endTime = time.clock()
    
    print("Total Runing time: %f"%(endTime - startTime))
    
    print(" Data Collected ")
    
if __name__ == '__main__':
    main()

Movie Reviews Collected
Candidates Collected
Got error {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.
